In [46]:
from sklearn.model_selection import train_test_split
import pandas as pd
import lightgbm as lgb

In [47]:
import re

In [48]:
from sklearn.metrics import mean_absolute_error


In [49]:
df_NTC = pd.read_csv("../../data/NTC.csv")
df_losses = pd.read_csv("../../data/Avtice-losses.csv")

df_renewable = pd.read_csv("../../data/Forecast-renewable-generation.csv")
df_temp = pd.read_csv("../../data/Forecast-temperature.csv")

df_losses = df_losses.drop(df_losses.index[0])

In [50]:
#print(df_NTC.columns)
print(df_losses.columns)
#print(df_renewable.columns)
print(df_temp.columns)

Index(['Unnamed: 0', 'Wirkverluste/Active Losses'], dtype='object')
Index(['datetime', 'temperature_fore_ch', 'temperature_fore_fr',
       'temperature_fore_de', 'temperature_fore_it'],
      dtype='object')


In [51]:
df_losses = df_losses.rename(columns={
    'Unnamed: 0': 'datetime',
})

In [52]:

df_NTC['datetime'] = pd.to_datetime(df_NTC['datetime'])
df_losses['datetime'] = pd.to_datetime(df_losses['datetime'])
df_renewable['datetime'] = pd.to_datetime(df_renewable['datetime'])
df_temp['datetime'] = pd.to_datetime(df_temp['datetime'])

In [53]:
df_losses['Wirkverluste/Active Losses'] = pd.to_numeric(df_losses['Wirkverluste/Active Losses'], errors='coerce')

In [54]:
# Convert "Zeitstempel" column to datetime format, if not already
df_losses['datetime'] = pd.to_datetime(df_losses['datetime'])

# Subtract 15 minutes from each date in the "Zeitstempel" column
df_losses['datetime'] = df_losses['datetime'] - pd.Timedelta(minutes=15)



In [55]:

df_losses_new = df_losses.set_index('datetime')

hourly_loss = df_losses_new.resample('1H', closed='left', label='left').sum()

In [56]:
hourly_loss["Wirkverluste/Active Losses"]=hourly_loss["Wirkverluste/Active Losses"]/1000


In [57]:
#hourly_loss

In [58]:


# Check if the data is already in 6-hour intervals
if (df_temp['datetime'].diff().dt.total_seconds() / 3600).median() == 6:
    # If the data is in 6-hour intervals, leave the datetime column as it is
    hourly_temp = df_temp
else:
    # If the data is not in 6-hour intervals, resample it to hourly intervals
    hourly_temp = df_temp.resample('H', on='datetime').mean().reset_index()

    # Fill missing values in other columns with the values from the initial 6-hourly data
    columns_to_fill = hourly_temp.columns.difference(['datetime'])
    hourly_temp[columns_to_fill] = hourly_temp[columns_to_fill].fillna(method='ffill', limit=6)
    # Replace '6' with the appropriate number if you want to fill more or fewer hours

# Print the resulting hourly data
display(hourly_temp.head(10))

,datetime,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it
0,2019-01-01 01:00:00,4.1067,5.9729,7.4268,4.0281
1,2019-01-01 02:00:00,4.1067,5.9729,7.4268,4.0281
2,2019-01-01 03:00:00,4.1067,5.9729,7.4268,4.0281
3,2019-01-01 04:00:00,4.1067,5.9729,7.4268,4.0281
4,2019-01-01 05:00:00,4.1067,5.9729,7.4268,4.0281
5,2019-01-01 06:00:00,4.1067,5.9729,7.4268,4.0281
6,2019-01-01 07:00:00,1.7595,5.5398,6.3425,2.7281
7,2019-01-01 08:00:00,1.7595,5.5398,6.3425,2.7281
8,2019-01-01 09:00:00,1.7595,5.5398,6.3425,2.7281
9,2019-01-01 10:00:00,1.7595,5.5398,6.3425,2.7281


In [59]:
print(df_NTC.columns)
print(hourly_loss.columns)
print(df_renewable.columns)
print(hourly_temp.columns)

Index(['datetime', 'CH_AT', 'CH_DE', 'CH_FR', 'CH_IT', 'AT_CH', 'DE_CH',
       'FR_CH', 'IT_CH'],
      dtype='object')
Index(['Wirkverluste/Active Losses'], dtype='object')
Index(['datetime', 'solar_fore_de [MW]', 'solar_fore_it [MW]',
       'wind_fore_de [MW]', 'wind_fore_it [MW]'],
      dtype='object')
Index(['datetime', 'temperature_fore_ch', 'temperature_fore_fr',
       'temperature_fore_de', 'temperature_fore_it'],
      dtype='object')


In [60]:
hourly_loss = hourly_loss.reset_index()

In [61]:
print(hourly_loss)

                 datetime  Wirkverluste/Active Losses
0     2019-01-01 00:00:00                  139.525004
1     2019-01-01 01:00:00                  129.716036
2     2019-01-01 02:00:00                  133.398074
3     2019-01-01 03:00:00                  135.133852
4     2019-01-01 04:00:00                  131.699424
...                   ...                         ...
26299 2021-12-31 19:00:00                  171.707318
26300 2021-12-31 20:00:00                  159.462903
26301 2021-12-31 21:00:00                  155.109520
26302 2021-12-31 22:00:00                  171.370277
26303 2021-12-31 23:00:00                  146.054791

[26304 rows x 2 columns]


In [62]:
merged_df = df_NTC.merge(hourly_loss, on='datetime', how='outer')
merged_df = merged_df.merge(df_renewable, on='datetime', how='outer')
merged_df = merged_df.merge(hourly_temp, on='datetime', how='outer')

In [63]:
na_counts_per_column = merged_df.isna().sum()

In [64]:
na_counts_per_column 

datetime                         0
CH_AT                            3
CH_DE                            3
CH_FR                            3
CH_IT                            3
AT_CH                            3
DE_CH                            3
FR_CH                            3
IT_CH                            3
Wirkverluste/Active Losses       0
solar_fore_de [MW]               3
solar_fore_it [MW]               3
wind_fore_de [MW]                3
wind_fore_it [MW]                3
temperature_fore_ch           1578
temperature_fore_fr           1578
temperature_fore_de           1578
temperature_fore_it           1578
dtype: int64

In [65]:
merged_df_incomplete = merged_df.dropna()

In [66]:
merged_df_incomplete.columns

Index(['datetime', 'CH_AT', 'CH_DE', 'CH_FR', 'CH_IT', 'AT_CH', 'DE_CH',
       'FR_CH', 'IT_CH', 'Wirkverluste/Active Losses', 'solar_fore_de [MW]',
       'solar_fore_it [MW]', 'wind_fore_de [MW]', 'wind_fore_it [MW]',
       'temperature_fore_ch', 'temperature_fore_fr', 'temperature_fore_de',
       'temperature_fore_it'],
      dtype='object')

In [67]:
#merged_df_incomplete.reset_index(inplace=True)
#merged_df_incomplete=merged_df_incomplete.drop('datetime')

In [68]:
merged_df_incomplete=merged_df_incomplete.rename(columns={'Wirkverluste/Active Losses': 'Loss'})

In [69]:
merged_df_incomplete=merged_df_incomplete.rename(columns={'solar_fore_de [MW]': 'solar_fore_de'})
merged_df_incomplete=merged_df_incomplete.rename(columns={'solar_fore_it [MW]': 'solar_fore_it'})
merged_df_incomplete=merged_df_incomplete.rename(columns={'wind_fore_de [MW]': 'wind_fore_de'})
merged_df_incomplete=merged_df_incomplete.rename(columns={'wind_fore_it [MW]': 'wind_fore_it'})
#drop the datetime

#merged_df_incomplete=merged_df_incomplete.drop('datetime')


In [70]:
merged_df_incomplete = merged_df_incomplete.reset_index()

In [71]:
#merged_df_incomplete = merged_df_incomplete.drop(columns=['datetime'])


In [72]:
merged_df_incomplete

,index,datetime,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH,Loss,solar_fore_de,solar_fore_it,wind_fore_de,wind_fore_it,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it
0,1,2019-01-01 01:00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,129.716036,0.0,0.0,23052.3310,4596.5916,4.1067,5.9729,7.4268,4.0281
1,2,2019-01-01 02:00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,133.398074,0.0,0.0,24969.9701,4478.5564,4.1067,5.9729,7.4268,4.0281
2,3,2019-01-01 03:00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,135.133852,0.0,0.0,27082.9626,4323.3712,4.1067,5.9729,7.4268,4.0281
3,4,2019-01-01 04:00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,131.699424,0.0,0.0,26890.9717,4231.8283,4.1067,5.9729,7.4268,4.0281
4,5,2019-01-01 05:00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,147.391128,0.0,0.0,27740.1555,4266.3082,4.1067,5.9729,7.4268,4.0281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24727,26305,2021-12-31 19:00:00,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,171.707318,0.0,0.0,36997.7200,1108.4000,8.5300,9.9600,10.7900,9.5900
24728,26306,2021-12-31 20:00:00,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,159.462903,0.0,0.0,35666.9300,1077.9700,8.0000,9.4000,10.6300,9.1100
24729,26307,2021-12-31 21:00:00,1200.0,4000.0,1400.0,3953.0,1200.0,800.0,3200.0,1810.0,155.109520,0.0,0.0,34383.8800,1048.2800,7.5000,8.8800,10.5100,8.6700
24730,26308,2021-12-31 22:00:00,1200.0,4000.0,1400.0,3780.0,1200.0,800.0,3200.0,1810.0,171.370277,0.0,0.0,33075.2500,1078.7800,6.9700,8.5100,10.3200,8.1400


## Create 24 different dataframes, each for a time of the day

#### create the 24 testing datasets

In [73]:
merged_df_incomplete['hour'] = merged_df_incomplete['datetime'].dt.hour

#### create all the lags

In [74]:
merged_ready_1= merged_df_incomplete

In [75]:
features = ['CH_DE', 'CH_FR', 'CH_IT', 'AT_CH', 'DE_CH', 'FR_CH', 'IT_CH',
               'solar_fore_de', 'solar_fore_it', 'wind_fore_de', 'wind_fore_it',
               'temperature_fore_ch', 'temperature_fore_fr', 'temperature_fore_de',
               'temperature_fore_it','Loss']

    # Create the lagged features
for feature in features:
    for lag in range(1, 336):
        col_name = f"{feature}_lag{lag}"
        merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)

merged_ready_1 = merged_ready_1.iloc[167:, :]

/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

...

/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_144011/2848475783.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

In [76]:
merged_ready_1

,index,datetime,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH,...,Loss_lag326,Loss_lag327,Loss_lag328,Loss_lag329,Loss_lag330,Loss_lag331,Loss_lag332,Loss_lag333,Loss_lag334,Loss_lag335
167,168,2019-01-08 00:00:00,700.0,4000.0,1200.0,1074.0,1200.0,800.0,3000.0,1910.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,169,2019-01-08 01:00:00,700.0,4000.0,1200.0,1074.0,1200.0,800.0,3000.0,1910.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,170,2019-01-08 02:00:00,700.0,4000.0,1200.0,1074.0,1200.0,800.0,3000.0,1910.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,171,2019-01-08 03:00:00,700.0,4000.0,1200.0,1097.0,1200.0,800.0,3000.0,1910.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,172,2019-01-08 04:00:00,700.0,4000.0,1200.0,1097.0,1200.0,800.0,3000.0,1910.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24727,26305,2021-12-31 19:00:00,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,...,123.168052,123.422885,121.685786,109.226337,114.119449,107.316018,88.716210,95.708585,87.365105,67.232117
24728,26306,2021-12-31 20:00:00,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,...,143.167697,123.168052,123.422885,121.685786,109.226337,114.119449,107.316018,88.716210,95.708585,87.365105
24729,26307,2021-12-31 21:00:00,1200.0,4000.0,1400.0,3953.0,1200.0,800.0,3200.0,1810.0,...,166.491003,143.167697,123.168052,123.422885,121.685786,109.226337,114.119449,107.316018,88.716210,95.708585
24730,26308,2021-12-31 22:00:00,1200.0,4000.0,1400.0,3780.0,1200.0,800.0,3200.0,1810.0,...,147.818352,166.491003,143.167697,123.168052,123.422885,121.685786,109.226337,114.119449,107.316018,88.716210


In [77]:
rename_dict = {
    'CH_AT': 'CH_AT_lag0',
    'CH_DE': 'CH_DE_lag0',
    'CH_FR': 'CH_FR_lag0',
    'CH_IT': 'CH_IT_lag0',
    'AT_CH': 'AT_CH_lag0',
    'DE_CH': 'DE_CH_lag0',
    'FR_CH': 'FR_CH_lag0',
    'IT_CH': 'IT_CH_lag0',
    #'Loss': 'Loss_lag0',
    'solar_fore_de': 'solar_fore_de_lag0',
    'solar_fore_it': 'solar_fore_it_lag0',
    'wind_fore_de': 'wind_fore_de_lag0',
    'wind_fore_it': 'wind_fore_it_lag0',
    'temperature_fore_ch': 'temperature_fore_ch_lag0',
    'temperature_fore_fr': 'temperature_fore_fr_lag0',
    'temperature_fore_de': 'temperature_fore_de_lag0',
    'temperature_fore_it': 'temperature_fore_it_lag0'#,
    #'hour': 'hour_lag0'
}

# Rename the columns
merged_ready_1 = merged_ready_1.rename(columns=rename_dict)

In [78]:
num_cols_per_line = 5
for i in range(0, len(merged_ready_1.columns), num_cols_per_line):
    print(', '.join(merged_ready_1.columns[i:i+num_cols_per_line]))

index, datetime, CH_AT_lag0, CH_DE_lag0, CH_FR_lag0
CH_IT_lag0, AT_CH_lag0, DE_CH_lag0, FR_CH_lag0, IT_CH_lag0
Loss, solar_fore_de_lag0, solar_fore_it_lag0, wind_fore_de_lag0, wind_fore_it_lag0
temperature_fore_ch_lag0, temperature_fore_fr_lag0, temperature_fore_de_lag0, temperature_fore_it_lag0, hour
CH_DE_lag1, CH_DE_lag2, CH_DE_lag3, CH_DE_lag4, CH_DE_lag5
CH_DE_lag6, CH_DE_lag7, CH_DE_lag8, CH_DE_lag9, CH_DE_lag10
CH_DE_lag11, CH_DE_lag12, CH_DE_lag13, CH_DE_lag14, CH_DE_lag15
CH_DE_lag16, CH_DE_lag17, CH_DE_lag18, CH_DE_lag19, CH_DE_lag20
CH_DE_lag21, CH_DE_lag22, CH_DE_lag23, CH_DE_lag24, CH_DE_lag25
CH_DE_lag26, CH_DE_lag27, CH_DE_lag28, CH_DE_lag29, CH_DE_lag30
CH_DE_lag31, CH_DE_lag32, CH_DE_lag33, CH_DE_lag34, CH_DE_lag35
CH_DE_lag36, CH_DE_lag37, CH_DE_lag38, CH_DE_lag39, CH_DE_lag40
CH_DE_lag41, CH_DE_lag42, CH_DE_lag43, CH_DE_lag44, CH_DE_lag45
CH_DE_lag46, CH_DE_lag47, CH_DE_lag48, CH_DE_lag49, CH_DE_lag50
CH_DE_lag51, CH_DE_lag52, CH_DE_lag53, CH_DE_lag54, CH_DE_lag55
CH

In [79]:
#make sure all the losses with lag are named -1


def modify_col_name(col_name):
    match = re.match(r"Loss_lag(\d+)", col_name)
    if match:
        number = int(match.group(1))
        return f"Loss_lag{number - 1}"
    return col_name

# Apply the function to each column name
merged_ready_1.columns = [modify_col_name(col) for col in merged_ready_1.columns]


In [80]:
# replac 0 with 24 because forecast 1-24
merged_ready_1.loc[merged_ready_1["hour"] == 0, "hour"] = 24



#### create the different df

In [81]:
dfs_total = {}

dataset_lags = [24, 48, 72, 96]

for i in dataset_lags:
    working_df = merged_ready_1.copy()
    
    threshold = i-1

    # Extract columns to drop based on their name
    cols_to_drop = []
    for col in working_df.columns:
        match = re.search(r'lag(\d+)', col)
        if match:
            lag_number = int(match.group(1))
            if lag_number > threshold:
                cols_to_drop.append(col)

    # Drop these columns
    working_df = working_df.drop(columns=cols_to_drop)
    
    
    
    dfs_total[f'df_{i}'] = working_df



In [82]:
for key, value in dfs_total.items():
    print(f"Key: {key}, Type: {type(value)}")


Key: df_24, Type: <class 'pandas.core.frame.DataFrame'>
Key: df_48, Type: <class 'pandas.core.frame.DataFrame'>
Key: df_72, Type: <class 'pandas.core.frame.DataFrame'>
Key: df_96, Type: <class 'pandas.core.frame.DataFrame'>


In [83]:
#  delete all the columns with lag lower than threshold

def set_midnight(df_input, threshold):
   
    
    working_df = df_input.copy()

    
    cols_to_drop = []
    for col in working_df.columns:
        match = re.search(r'lag(\d+)', col)
        if match:
            lag_number = int(match.group(1))
            if lag_number < threshold:
                cols_to_drop.append(col)
    working_df = working_df.drop(columns=cols_to_drop)
    
    return working_df

# Example usage:
# result_df = drop_lagged_columns(merged_ready_1, threshold_value)

In [84]:
list(dfs_total.keys())

['df_24', 'df_48', 'df_72', 'df_96']

In [85]:
dataset_lags

[24, 48, 72, 96]

In [86]:
filtered_datasets = {}
for lag in dataset_lags:
    key = f"df_{lag}"
    df_current = dfs_total[key]
    filtered_datasets[key] = {}
    
    for i in range(1, 25):
        filtered_df = df_current[df_current['hour'] == i]
        filtered_df_filtered=set_midnight(filtered_df, i)
        filtered_datasets[key][i] = filtered_df_filtered



In [87]:
filtered_datasets["df_72"][3].head(2)

,index,datetime,Loss,hour,CH_DE_lag3,CH_DE_lag4,CH_DE_lag5,CH_DE_lag6,CH_DE_lag7,CH_DE_lag8,...,Loss_lag62,Loss_lag63,Loss_lag64,Loss_lag65,Loss_lag66,Loss_lag67,Loss_lag68,Loss_lag69,Loss_lag70,Loss_lag71
170,171,2019-01-08 03:00:00,185.935455,3,4000.0,4000.0,4000.0,4000.0,4000.0,4000.0,...,164.544605,159.371671,182.746698,197.434057,197.096809,183.868235,199.355419,191.840655,184.524079,171.940970
194,195,2019-01-09 03:00:00,188.856706,3,4000.0,4000.0,4000.0,4000.0,4000.0,4000.0,...,170.252436,168.119765,194.269492,228.351993,257.248554,262.511493,226.068474,202.920325,190.839731,254.044293


In [91]:


def compute_mae(merged_df_ready_prep):
    # Ensure the merged_ready DataFrame is created first
    merged_ready = merged_df_ready_prep.copy()
    merged_ready=merged_ready.drop('datetime', axis=1)
   
    X = merged_ready.drop('Loss', axis=1)  # Features
    y = merged_ready['Loss']  # Target variable

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mae',
        'device': 'gpu',  # Enable GPU support
        
    }

    # Create the LightGBM dataset
    train_data = lgb.Dataset(X_train, label=y_train)
    
    # Train the model
    model = lgb.train(params, train_data)

    # Predict the test data
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    # Calculate MAE
    mae = mean_absolute_error(y_test, y_pred)

    return mae



In [100]:




# Lists to store the main keys, sub-keys and the number of rows
main_keys = []
sub_keys = []
num_rows = []
for lag in [48]:
#for lag in dataset_lags:
    dataset_key = "df_" + str(lag)
    if dataset_key in filtered_datasets:
        #for sub_key in range(1, 25):  # since you mentioned it goes from 1 to 24
        for sub_key in range(1, 25):
            main_keys.append(dataset_key)
            sub_keys.append(sub_key)
            num_rows.append(compute_mae(filtered_datasets[dataset_key][sub_key]))

# Create a DataFrame
result_df = pd.DataFrame({
    'Dataset_Key': main_keys,
    'Sub_Key': sub_keys,
    'Num_Rows': num_rows
})

print(result_df)



[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 116912
[LightGBM] [Info] Number of data points in the train set: 819, number of used features: 729
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 711 dense feature groups (0.56 MB) transferred to GPU in 0.195553 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 105.066014
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 100094
[LightGBM] [Info] Number of data points in the train set: 818, number of used features: 622
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 612 dense feature groups (0.48 MB) transferred to GPU in 0.008782 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 104.865695
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 83947
[LightGBM] [Info] Number of data points in the train set: 818, number of used features: 526
[Lig

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 68490
[LightGBM] [Info] Number of data points in the train set: 818, number of used features: 430
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 420 dense feature groups (0.33 MB) transferred to GPU in 0.008194 secs. 1 sparse feature groups
[LightGBM] [Info] Start 

In [102]:
#filename = "results_lags_[" + "_".join(map(str, dataset_lags)) + "].csv"
dataset_lags_custom=[48]
filename = "results_lags_[" + "_".join(map(str, dataset_lags_custom)) + "].csv"

In [103]:
filename

'results_lags_[48].csv'

In [104]:
result_df.to_csv(filename, index=False)

In [47]:
# to do: